In [ ]:
# DATA PARSER
import json
make_rulebook_data = json.load(open("RES_make_rulebook.json"))
use_rulebook_data = json.load(open("RES_use_rulebook.json"))
sparse_dense = json.load(open("RES_sparse_dense.json"))

In [ ]:
import matplotlib.pyplot as plt

# Function to create bar plots of times over voxel size and nonzero count over voxel size
def sparse_dense_bar_plot(times, nonzero_counts, voxel_sizes, xlabel='Voxel Size', ylabel='Value', title='Plot'):
    """
    Args:
    - times (list): List of time values for sparse/dense operations over different voxel sizes
    - nonzero_counts (list): List of nonzero counts for sparse/dense operations over different voxel sizes
    - voxel_sizes (list): List of voxel sizes
    - xlabel (str): Label for the x-axis (default is 'Voxel Size')
    - ylabel (str): Label for the y-axis (default is 'Value')
    - title (str): Title of the plot (default is 'Plot')

    Assumptions:
    - times, nonzero_counts, and voxel_sizes are lists of the same length
    - voxel_sizes represent the sizes of voxels used in the operations
    """

    # Generate x-axis positions for the bars
    x_positions = range(len(voxel_sizes))

    # Bar plot of times over voxel size
    plt.figure(figsize=(8, 6))
    plt.bar(x_positions, times, color='blue', width=0.5, tick_label=voxel_sizes)
    plt.xticks(fontsize=14)  # Change the font size of x-axis ticks
    plt.yticks(fontsize=14)  # Change the font size of y-axis ticks
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.title(title, fontsize=16)
    plt.show()

    # Bar plot of nonzero count over voxel size
    plt.figure(figsize=(8, 6))
    plt.xticks(fontsize=24)  # Change the font size of x-axis ticks
    plt.yticks(fontsize=24)  # Change the font size of y-axis ticks
    plt.bar(x_positions, nonzero_counts, color='green', width=0.5, tick_label=voxel_sizes)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.title(title, fontsize=16)
    plt.show()


# Function to create line plots of times over voxel size for different kernel sizes
def make_rulebook_line_plot(times_per_kernel, voxel_sizes, xlabel='Voxel Size', ylabel='Time', title='Plot'):
    """
    Args:
    - times_per_kernel (dict): Dictionary where keys are kernel sizes and values are lists of time values
    - voxel_sizes (list): List of voxel sizes
    - xlabel (str): Label for the x-axis (default is 'Voxel Size')
    - ylabel (str): Label for the y-axis (default is 'Time')
    - title (str): Title of the plot (default is 'Plot')

    Assumptions:
    - times_per_kernel is a dictionary where each key represents a kernel size and its value is a list of times
    - voxel_sizes represent the sizes of voxels used in the operations
    """

    plt.figure(figsize=(8, 6))
    for kernel_size, times in times_per_kernel.items():
        plt.plot(voxel_sizes, times, label=f'Kernel Size {kernel_size}', marker='o', linewidth=2)

    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.title(title, fontsize=16)
    plt.xscale('log', base=2)
    plt.xticks(fontsize=14)  # Change the font size of x-axis ticks
    plt.yticks(fontsize=14)  # Change the font size of y-axis ticks

    plt.legend()
    plt.show()

# Function to create line plots of times over voxel size for different convolution types and kernel sizes
def use_rulebook_line_plot(convolution_types, times_per_type, voxel_sizes, xlabel='Voxel Size', ylabel='Time', title='Plot'):
    """
    Args:
    - convolution_types (list): List of convolution types
    - times_per_type (dict): Dictionary where keys are convolution types and values are dictionaries
                            with kernel sizes as keys and lists of time values as values
    - voxel_sizes (list): List of voxel sizes
    - xlabel (str): Label for the x-axis (default is 'Voxel Size')
    - ylabel (str): Label for the y-axis (default is 'Time')
    - title (str): Title of the plot (default is 'Plot')

    Assumptions:
    - convolution_types is a list of different convolution types
    - times_per_type is a dictionary where each key represents a convolution type,
      and its value is a dictionary with kernel sizes as keys and lists of times as values
    - voxel_sizes represent the sizes of voxels used in the operations
    """

    for kernel_size in [3, 7]:
        plt.figure(figsize=(8, 6))
        for conv_type in convolution_types:
            plt.plot(voxel_sizes, times_per_type[conv_type][kernel_size], label=f'{conv_type}', marker='o', linewidth=2)
        plt.xticks(fontsize=14)  # Change the font size of x-axis ticks
        plt.yticks(fontsize=14)  # Change the font size of y-axis ticks

        plt.xlabel(xlabel, fontsize=16)
        plt.ylabel(ylabel, fontsize=16)
        plt.xscale('log', base=2)
        plt.title(f'{title}\nKernel Size {kernel_size}', fontsize=16)
        plt.legend()
        plt.show()


# MAKE RULEBOOKS
make_rb_data = make_rulebook_data["uni_rulebook"]
times_per_kernel = dict([k, [t[1] for t in k_data["times"]]] for k, k_data in make_rb_data)
voxel_sizes = make_rb_data[0][1]["sizes"]
print(times_per_kernel, voxel_sizes)
make_rulebook_line_plot(times_per_kernel, voxel_sizes, title="Time for Making a Rulebook For Block Size")

# FORWARD
conv_types = [("For-Loop", "inplace"), ("Atomic", "atomic"), ("Dense Convolution", "conv")]
times_per_type = dict([(listed_type, {3: [d["time_fore"] for d in use_rulebook_data[conv_type][0][1]["conv_times"]], 7: [d["time_fore"] for d in use_rulebook_data[conv_type][2][1]["conv_times"]]}) for listed_type, conv_type in conv_types])
conv_types = ["For-Loop", "Atomic", "Dense Convolution"]
use_rulebook_line_plot(conv_types, times_per_type, voxel_sizes, title="Forward Convolution Times")

#BACKWARD
conv_types = [("For-Loop", "inplace"), ("Atomic", "atomic"), ("Dense Convolution", "conv")]
times_per_type = dict([(listed_type, {3: [d["time_back"] for d in use_rulebook_data[conv_type][0][1]["conv_times"]], 7: [d["time_back"] for d in use_rulebook_data[conv_type][2][1]["conv_times"]]}) for listed_type, conv_type in conv_types])
conv_types = ["For-Loop", "Atomic", "Dense Convolution"]
use_rulebook_line_plot(conv_types, times_per_type, voxel_sizes, title="Backward Convolution Times")


# MAKE SPARSE
voxel_sizes = sparse_dense["sparse"]["sizes"]
nonzero_counts = [s / d for s, d in sparse_dense["sparse"]["on_count"]]
times = sparse_dense["sparse"]["times"]
sparse_dense_bar_plot(times, nonzero_counts, voxel_sizes)

# MAKE DENSE
voxel_sizes = sparse_dense["dense"]["sizes"]
nonzero_counts = [s / d for s, d in sparse_dense["dense"]["on_count"]]
times = sparse_dense["dense"]["times"]
sparse_dense_bar_plot(times, nonzero_counts, voxel_sizes)

In [ ]:
import matplotlib.pyplot as plt

# Data
x = [16, 32, 64, 128]
y = [0.091796875, 0.063568115234375, 0.04146575927734375, 0.030945777893066406]

plt.figure(figsize=(8, 5))

# Plotting the line with dots
plt.plot(x, y, marker='o', linestyle='-')
for i, txt in enumerate(y):
    plt.annotate(f'{txt:.3f}', (x[i], y[i]), textcoords="offset points", xytext=(0,10), ha='center')

# Labels and title
plt.xlabel('Voxel Size', fontsize=16)
plt.ylabel('Sparsity Percentage', fontsize=16)
plt.title('Sparsity at Size', fontsize=18)
plt.xscale("log", base=2)
plt.ylim(0, 0.1)
plt.xticks(fontsize=14)  # Change the font size of x-axis ticks
plt.yticks(fontsize=14)  # Change the font size of y-axis ticks

plt.show()
